In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

In [ ]:
# Day 1 - Points
# data source
# https://github.com/jokecamp/FootballData/blob/master/other/stadiums-with-GPS-coordinates.csv

# Read the CSV file
df = pd.read_csv("data/stadiums-with-GPS-coordinates.csv")

# Create a GeoDataFrame from the CSV data
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Draw the map background
fig = plt.figure(figsize=(8, 8), dpi=150)
m = Basemap(projection='lcc', resolution='f',
            width=0.8E6, height=1.1E6, 
            lat_0=54.4, lon_0=-3,)
m.etopo(scale=1, alpha=0.5)

# Scatter stadium data with size reflecting capacity
m.scatter(gdf.Longitude, gdf.Latitude, latlon=True,
          c='goldenrod', s=(30*gdf.Capacity / gdf.Capacity.max())**2,
          alpha=0.7, edgecolors='orangered')

# Make legend with dummy points
for a in [10000, 30000, 50000, 70000]:
    plt.scatter([], [], c='k', alpha=0.7, s=(30*a / gdf.Capacity.max())**2,
                label=str(int(a/1000)) + 'k')
plt.legend(title='Capacity', scatterpoints=1, frameon=True,
           labelspacing=1.2, loc='lower left');
plt.title("UK Football Stadium Capacity", fontdict={'family': 'serif', 'size': 16})

# Add large cities to map
# Add a point with a label
cities = [[-0.1278, 51.5074, '  London'],
          [-2.244644, 53.483959, '  Manchester'],
          [-1.898575, 52.489471, '  Birmingham'],
          [-4.251433, 55.860916, '  Glasgow'],
          [-1.600000, 54.966667, '  Newcastle'],]

for city in cities:
    # Convert lat/lon to map coordinates
    x, y = m(city[0], city[1])
    m.plot(x, y, '.', c='black', markersize=8)
    plt.text(x, y, city[2], fontsize=14, alpha=1, c='black', verticalalignment='center')

x, y, arrow_length = 0.9, 0.9, 0.1
plt.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords='axes fraction')

# save figure
plt.savefig('maps/1_points.png', dpi=300, bbox_inches='tight')